# Import

## Library import

In [1]:
import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
import glob
import numpy as np
import matplotlib.pyplot as plt

# Analysis

## Hector

### Policies
Importing, merging, and creating a column containing the state of policies.

In [2]:
policies_path = ".\\CCI_C-19_Policies\\data_tables\\policy_data\\table_data\\Current\\"
policies_files = glob.glob(policies_path + "*.csv")

content = [] #store contents from files

for filepath in policies_files:
    
    df = pd.read_csv(filepath, index_col=None)
    #Stripping the string to store the name of the file to a State column
    #State column should be the second column
    df.insert(1,"State", filepath.replace(policies_path,"").replace("_policy.csv",""))
    content.append(df)


policies_df = pd.concat(content)

In [3]:
policies_df

,date,State,policy,Restrict/Close,Opening (State),Deferring to County,Testing,Education,Health/Medical,Emergency Level,Transportation,Budget,Social Distancing,Other,Vaccine,Opening (County)
0,6-Mar-20,Alabama,Governor Ivey announced the formation of a Cor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
1,10-Mar-20,Alabama,Governor Ivey sent a memo to state agency head...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
2,13-Mar-20,Alabama,Governor Ivey issued a state of emergency for ...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Y,NaN,NaN
3,14-Mar-20,Alabama,Governor Ivey closed all Alabama public school...,Y,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15-Mar-20,Alabama,Governor Ivey authorized directors of all stat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,29-Apr-21,Wyoming,Wyoming’s two remaining COVID-19 statewide pub...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
75,13-May-21,Wyoming,Governor Gordon Withdraws Wyoming from COVID-e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN
76,21-May-21,Wyoming,The Wyoming Department of Health (WDH) is anno...,NaN,Y,NaN,NaN,Y,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN
77,4-Aug-21,Wyoming,"Governor Will Not Issue Mask Mandate, Stresses...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


In [4]:
# Getting all unique values from the columns after "policy" columns in the dataframe
for col in policies_df.columns[3:]:
    print(col,": ",set(policies_df[col]))

Restrict/Close :  {nan, ' ', 'Y', 'y'}
Opening (State) :  {nan, 'Y', 'y'}
Deferring to County :  {nan, 'Y'}
Testing :  {nan, 'Y'}
Education :  {nan, 'Y'}
Health/Medical :  {nan, 'Y', 'y'}
Emergency Level :  {nan, 'Y', 'y'}
Transportation :  {nan, 'Y'}
Budget :  {nan, 'Y', 'y'}
Social Distancing :  {nan, 'Y', 'y'}
Other :  {nan, 'Y', 'y'}
Vaccine :  {nan, 'Y', 'y'}
Opening (County) :  {nan, 'Y'}


In [5]:
# Fixin some of the inconsistencies such as lower case Y seen in
# previous cell.

# Replacing only single lowercase y to Y
policies_df.replace(to_replace=r'^y$',value="Y",regex=True,inplace=True)

In [6]:
# Finding which entry has a whitespace
policies_df[policies_df['Restrict/Close'].str.contains("\s", regex=True, na=False)]

,date,State,policy,Restrict/Close,Opening (State),Deferring to County,Testing,Education,Health/Medical,Emergency Level,Transportation,Budget,Social Distancing,Other,Vaccine,Opening (County)
25,11-Apr-20,Kansas,The Kansas Supreme Court upheld the Governor'...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


In [7]:
# Reading the policy from entry
list(policies_df["policy"][policies_df['Restrict/Close'].str.contains("\s", regex=True, na=False)])

[" The Kansas Supreme Court upheld the Governor's executive order limiting the size of mass gathering to 10 people, including that of church services and funerals. "]

Policy doesnt provide enough info so will just fill with NaN in the meantime.

In [8]:
policies_df['Restrict/Close'].replace("\s", np.nan ,inplace=True,regex=True)

In [9]:
# Re-verifying all unique values
for col in policies_df.columns[3:]:
    print(col,": ",set(policies_df[col]))

Restrict/Close :  {nan, 'Y'}
Opening (State) :  {nan, 'Y'}
Deferring to County :  {nan, 'Y'}
Testing :  {nan, 'Y'}
Education :  {nan, 'Y'}
Health/Medical :  {nan, 'Y'}
Emergency Level :  {nan, 'Y'}
Transportation :  {nan, 'Y'}
Budget :  {nan, 'Y'}
Social Distancing :  {nan, 'Y'}
Other :  {nan, 'Y'}
Vaccine :  {nan, 'Y'}
Opening (County) :  {nan, 'Y'}


In [10]:
# Dropping, if any, duplicates that are contained from the first 3 cols
policies_df.drop_duplicates(["date","State","policy"], inplace=True)

In [11]:
# Checking dups of only first 2 cols
policies_df[policies_df.duplicated(["date","State"], keep= False)]

,date,State,policy,Restrict/Close,Opening (State),Deferring to County,Testing,Education,Health/Medical,Emergency Level,Transportation,Budget,Social Distancing,Other,Vaccine,Opening (County)
4,15-Mar-20,Alabama,Governor Ivey authorized directors of all stat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
5,15-Mar-20,Alabama,The Governor authorized directors of all state...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN
7,17-Mar-20,Alabama,The Governor emphasized measures taken by fina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
8,17-Mar-20,Alabama,The Alabama Department of Public Health issued...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,20-Mar-20,Alabama,The Governor issued an updated statewide publi...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,1-Jul-20,Wyoming,The Wyoming Superintendent announced the relea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
54,19-Nov-20,Wyoming,Governor Mark Gordon has authorized additional...,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,19-Nov-20,Wyoming,Governor Mark Gordon announced additional stat...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,11-Dec-20,Wyoming,Governor Mark Gordon announced the opening of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


In [12]:
list(policies_df.columns[2:])

['policy',
 'Restrict/Close',
 'Opening (State)',
 'Deferring to County',
 'Testing',
 'Education',
 'Health/Medical',
 'Emergency Level',
 'Transportation',
 'Budget',
 'Social Distancing',
 'Other',
 'Vaccine',
 'Opening (County)']

There seem to be multiple entries per date,state. Maybe merging them could be a good idea.

In [13]:
policies_df.dtypes

date                   object
State                  object
policy                 object
Restrict/Close         object
Opening (State)        object
Deferring to County    object
Testing                object
Education              object
Health/Medical         object
Emergency Level        object
Transportation         object
Budget                 object
Social Distancing      object
Other                  object
Vaccine                object
Opening (County)       object
dtype: object

In [14]:
# Combining data between the date and state columns
# Inspired by
# https://stackoverflow.com/questions/14529838/apply-multiple-functions-to-multiple-groupby-columns

cols = list(policies_df.columns[2:])
dic = {}

for col in cols:
    # Changin datatype to string to avoid issues with nans
    policies_df = policies_df.astype({col: str})
    # Property of joining with commas for the agg
    dic[col] = ",".join

policies_df = policies_df.groupby(["date","State"]).agg(dic).reset_index()


In [15]:
policies_df

,date,State,policy,Restrict/Close,Opening (State),Deferring to County,Testing,Education,Health/Medical,Emergency Level,Transportation,Budget,Social Distancing,Other,Vaccine,Opening (County)
0,1-Apr-20,American Samoa,The Governor issued the second amended declara...,Y,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1-Apr-20,Arizona,The Governor announced the contribution of $25...,"nan,nan,nan,nan","nan,nan,nan,nan","nan,nan,nan,nan","nan,nan,nan,nan","nan,nan,nan,nan","nan,nan,Y,nan","nan,nan,nan,nan","nan,nan,nan,nan","nan,nan,nan,nan","nan,nan,nan,nan","Y,Y,Y,Y","nan,nan,nan,nan","nan,nan,nan,nan"
2,1-Apr-20,Arkansas,The Governor implemented day use operations on...,"nan,nan","nan,nan","nan,nan","nan,Y","nan,nan","nan,nan","nan,nan","nan,nan","nan,nan","nan,nan","Y,Y","nan,nan","nan,nan"
3,1-Apr-20,California,The Governor issued an executive order that al...,"nan,nan","nan,nan","nan,nan","nan,nan","nan,Y","nan,nan","nan,nan","nan,nan","nan,nan","nan,nan","Y,nan","nan,nan","nan,nan"
4,1-Apr-20,Colorado,The Governor announced that he would extend th...,"Y,nan","nan,nan","nan,nan","nan,nan","nan,nan","nan,Y","nan,nan","nan,nan","nan,nan","nan,nan","nan,nan","nan,nan","nan,nan"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,9-Sep-20,New Hampshire,Governor Chris Sununu issued Emergency Order #...,nan,nan,nan,nan,nan,nan,Y,nan,nan,nan,Y,nan,nan
5081,9-Sep-20,New York,Governor Cuomo announced indoor dining in New ...,nan,Y,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5082,9-Sep-20,Oklahoma,The Oklahoma State Department of Health (OSDH)...,nan,Y,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5083,9-Sep-20,South Dakota,Governor Kristi Noem laid out a framework for ...,nan,nan,nan,nan,nan,nan,nan,nan,Y,nan,Y,nan,nan


In [20]:
#Iterating through dataframe to
#Fix 

# for index, row in policies_df.iterrows():
    
# #     for col in list(policies_df.columns)[3:]:#only the 4th column onwards
        
# #         for val in row[col]:
# #             splt = val.split(',') #split on comma
# #             print(val)
#     for val in row:
#         print(val)
#     #         if len(splt) > 1: #only if split contains more than 1 value
#     #             if 'Y' in splt: #Filtering only ones containing Y
#     #                 policies_df[index]

#     #             else: #
                

# for index, row in policies_df.iterrows():
#     print(row["date"])

def reducto(df):
    dic = {}
    
    cols = list(df.columns)[3:] #Only the columns 4th and forward
    
    for col in cols:
        for val in df[col]:
            split = val.split(',')
            if len(split) > 1:
                if "Y" in split:
                    dic[col] = "Y"
                else:
                    dic[col] = np.nan
    return pd.Series(dic, index=cols)

policies_df.groupby(["date","State","policy"]).apply(reducto).reset_index()  

C:\Users\jis\AppData\Local\Temp/ipykernel_12372/3467614195.py:36: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series(dic, index=cols)


,date,State,policy,Restrict/Close,Opening (State),Deferring to County,Testing,Education,Health/Medical,Emergency Level,Transportation,Budget,Social Distancing,Other,Vaccine,Opening (County)
0,1-Apr-20,American Samoa,The Governor issued the second amended declara...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1-Apr-20,Arizona,The Governor announced the contribution of $25...,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,Y,NaN,NaN
2,1-Apr-20,Arkansas,The Governor implemented day use operations on...,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
3,1-Apr-20,California,The Governor issued an executive order that al...,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
4,1-Apr-20,Colorado,The Governor announced that he would extend th...,Y,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,9-Sep-20,New Hampshire,Governor Chris Sununu issued Emergency Order #...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5081,9-Sep-20,New York,Governor Cuomo announced indoor dining in New ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5082,9-Sep-20,Oklahoma,The Oklahoma State Department of Health (OSDH)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5083,9-Sep-20,South Dakota,Governor Kristi Noem laid out a framework for ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Policies Dictionary

No column dictionary was provided in the repository for these. Will make my own.

**The following are the primary cols**

date: date it occured

State: string of State

policy: string of actions taken by the state

**The rest are just booleans, of which are mostly either Y or nan**

Restrict/Close: Closure of unesential businesses?

Opening (State): Opened the state to foreigners?

Deferring to County: ?

Testing: Covid testing began?

Education: Physical attendance of education again?

Health/Medical: ?

Emergency Level: ?

Transportation: Public transportation??
       
Budget: ?
       
Social Distancing: Social distancing was active?

Other: ?
       
Vaccine: Vaccines were active?

Opening (County): ?

#### Will attempt keyword extraction from the dataset